In [1]:
import pandas as pd

# schalchemy dependencies
from sqlalchemy import create_engine, MetaData
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Numeric, Text, Float, ForeignKey
from sqlalchemy.orm import sessionmaker, relationship

# Import PyMySQL (Not needed if mysqlclient is installed)
import pymysql
pymysql.install_as_MySQLdb()

In [2]:
# Import and establish Base for which classes will be constructed 

Base = declarative_base()

In [3]:
car_sales_df = pd.read_csv('../Resources/car_combined_test_train.csv')

car_df = car_sales_df.rename(columns={'Resale-Cat':'Resale_Cat','Index': 'index'})
car_df.head()

,index,Resale_Cat,fuel_type,vehicle_class,displ,cyl,trans,drive,smog_rating,city_mpg,hwy_mpg,cmb_mpg,Greenhouse_gas_score,smartway,comb_CO2,Sales_2016,Sales_2017,Sales_2018
0,0,Low,Gasoline,large car,3.0,6,SemiAuto-8,2WD,6,21.0,29.0,24.0,5,No,371.0,12071,8731,7909
1,1,Low,Gasoline,large car,3.0,6,SemiAuto-8,2WD,6,21.0,29.0,24.0,5,No,371.0,12071,8731,7909
2,2,Low,Gasoline,large car,4.4,8,SemiAuto-8,2WD,5,17.0,25.0,20.0,4,No,436.0,12071,8731,7909
3,3,Low,Gasoline,large car,4.4,8,SemiAuto-8,2WD,6,17.0,25.0,20.0,4,No,436.0,12071,8731,7909
4,4,Low,Gasoline,large car,4.4,8,SemiAuto-8,4WD,5,16.0,25.0,19.0,4,No,459.0,12071,8731,7909


In [4]:
#Reading the dataset
carvalidate_df = pd.read_csv('../Resources/car_data_combined_validation.csv')

carvalidate_df.head()

,Index,Resale_Cat,model,fuel_type,vehicle_class,displ,cyl,trans,drive,smog_rating,city_mpg,hwy_mpg,cmb_mpg,Greenhouse_gas_score,smartway,comb_CO2,Sales_2016,Sales_2017,Sales_2018
0,4,Low,BMW 750i,Gasoline,large car,4.4,8,SemiAuto-8,4WD,5,16,25,19,4,No,459,12071,8731,7909
1,65,Low,FIAT 500L,Gasoline,station wagon,1.4,4,Man-6,2WD,6,25,33,28,7,Yes,312,14380,11467,4916
2,119,Low,JAGUAR XJ,Gasoline,large car,3.0,6,SemiAuto-8,2WD,9,18,27,21,5,No,414,3542,2553,1348
3,132,Low,NISSAN Leaf,Electricity,midsize car,0.0,0,Auto-1,2WD,10,126,101,114,10,Elite,0,13251,10458,14565
4,139,High,TOYOTA 4Runner,Gasoline,standard SUV,4.0,6,SemiAuto-5,4WD,6,17,21,18,4,No,478,104456,119597,130025


In [5]:
car_sales_encoded = pd.read_csv("../Resources/car_sales_encoded.csv")

train_df = car_sales_encoded.rename(columns={'Resale-Cat':'Resale_Cat'})
train_df.head()

,Resale_Cat,fuel_type,vehicle_class,trans,drive,smog_rating,city_mpg,hwy_mpg,cmb_mpg,Greenhouse_gas_score,smartway,comb_CO2,Sales_2016,Sales_2017,Sales_2018
0,1,3,0,7,0,5,17.0,25.0,1,4,1,436.0,12071,8731,7909
1,1,3,0,7,0,6,17.0,25.0,1,4,1,436.0,12071,8731,7909
2,1,3,0,7,1,5,16.0,25.0,1,4,1,459.0,12071,8731,7909
3,0,3,2,2,0,6,20.0,27.0,1,5,1,397.0,103217,106583,20848
4,0,3,2,2,0,6,20.0,27.0,1,5,1,397.0,103217,106583,20848


In [6]:
test_df = pd.read_csv("../Resources/encoded_output.csv")

test_df

,Resale_Cat,fuel_type,vehicle_class,trans,drive,smog_rating,city_mpg,hwy_mpg,cmb_mpg,Greenhouse_gas_score,smartway,comb_CO2,Sales_2016,Sales_2017,Sales_2018
0,1,1,0,6,1,5,16,25,1,4,1,459,12071,8731,7909
1,1,1,5,4,0,6,25,33,2,7,2,312,14380,11467,4916
2,1,1,0,6,0,9,18,27,1,5,1,414,3542,2553,1348
3,1,0,1,0,0,10,126,101,0,10,0,0,13251,10458,14565
4,0,1,4,5,1,6,17,21,1,4,1,478,104456,119597,130025
5,0,1,2,3,0,5,16,22,1,3,1,497,207299,204211,30244
6,0,1,3,1,1,6,17,21,1,3,1,481,180991,179188,228293
7,0,1,2,2,0,6,19,26,1,5,1,410,23666,32068,28486


In [7]:
# Import modules to declare columns and column data types
from sqlalchemy import Column, Integer, String, Float

In [8]:
# Create All classes
# ----------------------------------
class Car(Base):
    __tablename__ = 'car'
    index = Column(Integer, primary_key=True)
    Resale_Cat = Column(String(255))
    model = Column(String(255))
    vehicle_class = Column(String(255))
    Fuel_type = Column(String(255))
    displ = Column(Float)
    cyl = Column(Integer)
    trans= Column(String(255))
    drive = Column(String(255))
    smog_rating = Column(Integer)
    city_mpg  = Column(Float)
    hwy_mpg  = Column(Float)
    cmb_mpg = Column(Float)
    Greenhouse_gas_score = Column(Integer)
    smartway = Column(String(255))
    comb_CO2 =Column(Float)
    Sales_2016 =Column(Integer)
    Sales_2017 = Column(Integer)
    Sales_2018=Column(Integer)


# ----------------------------------
class CarValidate(Base):
    __tablename__ = 'carvalidate'
    index = Column(Integer, primary_key=True)
    Resale_Cat = Column(String(255))
    model = Column(String(255))
    vehicle_class = Column(String(255))
    Fuel_type = Column(String(255))
    displ = Column(Float)
    cyl = Column(Integer)
    trans= Column(String(255))
    drive = Column(String(255))
    smog_rating = Column(Integer)
    city_mpg  = Column(Float)
    hwy_mpg  = Column(Float)
    cmb_mpg = Column(Float)
    Greenhouse_gas_score = Column(Integer)
    smartway = Column(String(255))
    comb_CO2 =Column(Float)
    Sales_2016 =Column(Integer)
    Sales_2017 = Column(Integer)
    Sales_2018=Column(Integer)

# ----------------------------------
class Train(Base):
    __tablename__ = 'train'
    id = Column(Integer, primary_key=True)
    Resale_Cat =Column(Integer)
    vehicle_class = Column(Integer)
    Fuel_type = Column(Integer)
    
    trans= Column(Integer)
    drive = Column(Integer)
    smog_rating = Column(Integer)
    city_mpg  = Column(Float)
    hwy_mpg  = Column(Float)
    cmb_mpg = Column(Integer)
    Greenhouse_gas_score = Column(Integer)
    smartway = Column(Integer)
    comb_CO2 =Column(Float)
    Sales_2016 =Column(Integer)
    Sales_2017 = Column(Integer)
    Sales_2018=Column(Integer)
# ----------------------------------
class Test(Base):
    __tablename__ = 'test'
    id = Column(Integer, primary_key=True)
    Resale_Cat =Column(Integer)
    vehicle_class = Column(Integer)
    Fuel_type = Column(Integer)
    
    trans= Column(Integer)
    drive = Column(Integer)
    smog_rating = Column(Integer)
    city_mpg  = Column(Float)
    hwy_mpg  = Column(Float)
    cmb_mpg = Column(Integer)
    Greenhouse_gas_score = Column(Integer)
    smartway = Column(Integer)
    comb_CO2 =Column(Float)
    Sales_2016 =Column(Integer)
    Sales_2017 = Column(Integer)
    Sales_2018=Column(Integer)

In [9]:
# Create Database Connection
# ----------------------------------
# Establish Connection to MySQL
engine = create_engine("sqlite:///fuel_economy.sqlite")
conn = engine.connect()

In [10]:
# Create Fuel table within the database
Base.metadata.create_all(conn)

In [11]:
car_df.to_sql("car", engine, if_exists='append', index=False)
carvalidate_df.to_sql("carvalidate", engine, if_exists='append', index=False)

In [12]:
train_df.to_sql("train", engine, if_exists='append', index=False)
test_df.to_sql("test", engine, if_exists='append', index=False)

In [13]:
# To push the objects made and query the server we use a 
#Session object
from sqlalchemy.orm import Session
session = Session(bind=engine)

In [14]:
engine.execute("SELECT * FROM car LIMIT 1").fetchall()

[(0, 'Low', None, 'large car', 'Gasoline', 3.0, 6, 'SemiAuto-8', '2WD', 6, 21.0, 29.0, 24.0, 5, 'No', 371.0, 12071, 8731, 7909)]

In [15]:
engine.execute("SELECT * FROM carvalidate LIMIT 1").fetchall()

[(4, 'Low', 'BMW 750i', 'large car', 'Gasoline', 4.4, 8, 'SemiAuto-8', '4WD', 5, 16.0, 25.0, 19.0, 4, 'No', 459.0, 12071, 8731, 7909)]

In [16]:
engine.execute("SELECT * FROM train LIMIT 1").fetchall()

[(1, 1, 0, 3, 7, 0, 5, 17.0, 25.0, 1, 4, 1, 436.0, 12071, 8731, 7909)]

In [17]:
engine.execute("SELECT * FROM test LIMIT 1").fetchall()

[(1, 1, 0, 1, 6, 1, 5, 16.0, 25.0, 1, 4, 1, 459.0, 12071, 8731, 7909)]

In [18]:
test_data = engine.execute("SELECT * FROM test").fetchall()

In [19]:
print(test_data)

[(1, 1, 0, 1, 6, 1, 5, 16.0, 25.0, 1, 4, 1, 459.0, 12071, 8731, 7909), (2, 1, 5, 1, 4, 0, 6, 25.0, 33.0, 2, 7, 2, 312.0, 14380, 11467, 4916), (3, 1, 0, 1, 6, 0, 9, 18.0, 27.0, 1, 5, 1, 414.0, 3542, 2553, 1348), (4, 1, 1, 0, 0, 0, 10, 126.0, 101.0, 0, 10, 0, 0.0, 13251, 10458, 14565), (5, 0, 4, 1, 5, 1, 6, 17.0, 21.0, 1, 4, 1, 478.0, 104456, 119597, 130025), (6, 0, 2, 1, 3, 0, 5, 16.0, 22.0, 1, 3, 1, 497.0, 207299, 204211, 30244), (7, 0, 3, 1, 1, 1, 6, 17.0, 21.0, 1, 3, 1, 481.0, 180991, 179188, 228293), (8, 0, 2, 1, 2, 0, 6, 19.0, 26.0, 1, 5, 1, 410.0, 23666, 32068, 28486)]


In [20]:
engine = create_engine("sqlite:///fuel_economy.sqlite")
conn = engine.connect()

In [21]:
df = pd.read_sql_query("SELECT * FROM test",conn)

In [22]:
df

,id,Resale_Cat,vehicle_class,Fuel_type,trans,drive,smog_rating,city_mpg,hwy_mpg,cmb_mpg,Greenhouse_gas_score,smartway,comb_CO2,Sales_2016,Sales_2017,Sales_2018
0,1,1,0,1,6,1,5,16.0,25.0,1,4,1,459.0,12071,8731,7909
1,2,1,5,1,4,0,6,25.0,33.0,2,7,2,312.0,14380,11467,4916
2,3,1,0,1,6,0,9,18.0,27.0,1,5,1,414.0,3542,2553,1348
3,4,1,1,0,0,0,10,126.0,101.0,0,10,0,0.0,13251,10458,14565
4,5,0,4,1,5,1,6,17.0,21.0,1,4,1,478.0,104456,119597,130025
5,6,0,2,1,3,0,5,16.0,22.0,1,3,1,497.0,207299,204211,30244
6,7,0,3,1,1,1,6,17.0,21.0,1,3,1,481.0,180991,179188,228293
7,8,0,2,1,2,0,6,19.0,26.0,1,5,1,410.0,23666,32068,28486


In [23]:
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pandas as pd
import os
import pickle

In [24]:
filename = '../Model/finalized_smartway_model_KN.sav'
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))

In [25]:
new_df = pd.read_sql_query("SELECT * FROM test", conn)

In [29]:
newX = new_df.drop(['id','smartway'], axis=1)
newy = new_df['smartway']
print(newy)

0    1
1    2
2    1
3    0
4    1
5    1
6    1
7    1
Name: smartway, dtype: int64


In [27]:
result = loaded_model.score(newX, newy)
ynew = loaded_model.predict(newX)

In [28]:
print(ynew)

[1 1 1 0 1 1 1 1]
